> **DO YOU USE GITHUB?**  
If True: print('Remember to make your edits in a personal copy of this notebook')  
Else: print('You don't have to understand. Continue your life.')

# Module 7: Web Scraping 2

In module_6 your learned some powerful tricks. Tricks that will work when the data is already shipped in a neat format. However this is often not the case. In this session we shall learn the art of parsing unstructured text and a more principled and advanced method of parsing HTML.

This will help you build ***custom datasets*** within just a few hours or days work, that would have taken ***months*** to curate and clean manually.



Readings for `module 6+7+8`:
- [Python for Data Analysis, chapter 6](https://bedford-computing.co.uk/learning/wp-content/uploads/2015/10/Python-for-Data-Analysis.pdf)
- [A Practical Introduction to Web Scraping in Python](https://realpython.com/python-web-scraping-practical-introduction/)
- [An introduction to web scraping with Python](https://towardsdatascience.com/an-introduction-to-web-scraping-with-python-a2601e8619e5)
- [Introduction to Web Scraping using Selenium](https://medium.com/the-andela-way/introduction-to-web-scraping-using-selenium-7ec377a8cf72)

Video materiale from `ISDS 2020`:
- [Web Scraping 1](https://bit.ly/ISDS2021_6)
- [Web Scraping 2](https://bit.ly/ISDS2021_7)
- [Web Scraping 3](https://bit.ly/ISDS2021_8)

Other ressources:
- [Nicklas Webpage](https://nicklasjohansen.netlify.app/)
- [Data Driven Organizational Analysis, Fall 2021](https://efteruddannelse.kurser.ku.dk/course/2021-2022/ASTK18379U)
- [Master of Science (MSc) in Social Data Science](https://www.socialdatascience.dk/education)


## Introduction to HTML
[What is HTML?](https://www.w3schools.com/whatis/whatis_html.asp)  

HTML has a Tree structure. 

Each node in the tree has:
- Children, siblings, parents, descendants. 
- Ids and attributes

<img src="http://www.openbookproject.net/tutorials/getdown/css/images/lesson4/HTMLDOMTree.png"/>


## Important syntax and patterns
_______________
```html 
<p>The p tag indicates a paragraph <p/>
```
_______________
```html 
<b>The b tag makes the text bold, giving us a clue to its importance </b>
```
output: <b>The b tag makes the text bold, giving us a clue to its importance </b>
```html 

<em>The em tag emphasize the text</em>, giving us a clue to its importance
```
output: <em>The em tag makes emphasize the text</em>, giving us a clue to its importance
___________
```html 
<h1>h1</h1><h2>h2</h2><h3>h3</h3><b>Headers give similar clues</b>
```
output:
<h1>h1</h1><h2>h2</h2><h3>h3</h3><b>Headers give similar clues</b>  
  
```html 
<a href="www.google.com">The a tag creates a hyperlink <a/>
```
output: <a href="www.google.com">The a tag creates a hyperlink <a/>

## How do we find our way around this tree?
1. Regex: Extracting string patterns using .split and regular expresssions
2. CSS-selectors: Specifying paths using css-selectors, xpath syntax.
3. ```BeautifulSoup```: A more powerful, principled and readable way to parse data and navigate HTML

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import selenium
import time
import pandas as pd

### Regex
- [What is regex?](https://en.wikipedia.org/wiki/Regular_expression)
- The brute force way is to parse by convering your downloded matriale into a large string
- Now you can create standard string operations
- And apply smart regex to identify the data you are looking for e.g. links.

In [ ]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
html = response.text
#html.split('\n')
#re.findall("(?P<url>https?://[^\s]+)", html)[0]

### CSS Selectors 
- [What is a CSS Selector?](https://www.w3schools.com/css/css_selectors.asp)
- Another way to browse through the HTML tree
- Define a unique path to an element in the HTML tree.
- It is quick but has to be hardcoded and also more likely to break.
- [Nicklas recommend using this free Google Chrome CSS Selector](https://chrome.google.com/webstore/detail/selectorgadget/mhjhnkcfbdhnjickkkdbjoemdmbfginb)

In [ ]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
soup.select('.dcr-125vfar')[0].text

## Parsing HTML with BeautifulSoup
BeautifulSoup makes the html tree navigable. 
It allows you to:
    * Search for elements by tag name and/or by attribute.
    * Iterate through them, go up, sideways or down the tree.
    * Furthermore it helps you with standard tasks such as extracting raw text from html,
    which would be a very tedious task if you had to hardcode it using `.split` commands and using your own regular expressions will be unstable.

In [ ]:
url = 'https://www.theguardian.com/us-news/2019/aug/14/taco-eating-contest-death-fresno-california'
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html,'lxml') # parse the raw html using BeautifoulSoup

In [ ]:
soup.find('h1')['class'][0].strip()

In [ ]:
# example: finding hyperlinks
links = soup.find_all('a') # find all a tags -connoting a hyperlink.
[link['href'] for link in links if link.has_attr('href')][0:5] # unpack the hyperlink from the a nodes.

In [ ]:
# example: finding headline
headline = soup.find('h1') # search for the first headline: h1 tag. 
name = headline['class'][0].strip() # use the class attribute name as column name.
value = headline.text.strip() # extract text using build in method.
print(name,':',value)

In [ ]:
# example: finding article_text
article_text = soup.findAll('div', {'class':'dcr-185kcx9'})[0]
article_text.text

In [ ]:
article_text.findAll(['a'])

Say we are interested in how articles cite sources to back up their story i.e. their hyperlink behaviour within the article, and we want to see if the media has changed their behaviour.

We know how to search for links. But the cool part is that we can search from anywhere in the HTML tree. This means that once we have located the article content node - as above - we can search from there. This results in hyperlinks used within the article text.


In [ ]:
# example: finding citation links
citations = article_text.findAll('a')

citation_links = [] # define container to the hyperlinks
for citation in citations: # iterate through each citation node
    if citation.has_attr('data-link-name'): # check if it has the right attribute
        if citation['data-link-name'] =='in body link': # and if the value of that attribute is correct
            print(citation['href'])
            citation_links.append(citation['href']) #  add link to the container

citation_links

## Creating a dataset from www.bold.dk

### Let's put together some of the stuff we have learned so far
1. **Investigate:** In this example we will try to investigate the website to uderstand its structure. 
2. **Mapping:** Then we will try to collect all the urls and save them into a list
3. **Parsing:** At last, we will try to collect the information in each url in a simpel loop.

#### First, we pay around with the site trying to understand its structure

In [ ]:
len(soup.find_all('div',{'class':'news_list_item'}))
articles[0].a['href']
type(articles)


In [ ]:
# define our URL
url = 'https://www.bold.dk/' 

# connects to site
response = requests.get(url)

# parse data with BeautifulSoup
soup = BeautifulSoup(response.text,'lxml') # parse the HTML

#identify articles to scrape by inspecting site
articles = soup.find_all('div',{'class':'news_list_item'}) # search for the ul node

# checking if articles match website
for i in range(len(articles)):
    print(articles[i].text.strip())

# identifying how to find an url from an article
article_url = articles[0].attrs['data-vr-contentbox-url']
print(article_url)

In [ ]:
articles[0]

#### Second, we create a list of urls that we want to scrape

In [ ]:
url = 'https://www.bold.dk/' 
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
articles = soup.find_all('div',{'class':'news_list_item'})

#create an empty list
list_of_article_urls = []

# creating a loop that appends the article url to the list above
for i in range(len(articles)):
    list_of_article_urls.append(articles[i].attrs['data-vr-contentbox-url'])

#printing the list
list_of_article_urls

#printing one example
#print(list_of_article_urls[0])

#### Third, we scrape each site from the url list

In [ ]:
# this step usually reuqiere a new step of investigation
# to figure out what information you want to download
# in this example we want the title, the lead and time posted

# creatig empty list for the infomation we want to extract for every article
h1_list = []
lead = []
time_posted = []

for i in range(len(list_of_article_urls)): # 10 #len(list_of_article_urls)
    
    # this time we scrape for each news article in the url list we created before
    url = list_of_article_urls[i]
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'lxml')
    
    # pedagogical way of append title to list
    temp_1 = soup.find_all('h1')
    temp_1 = temp_1[1]
    temp_1 = temp_1.text.strip()
    h1_list.append(temp_1)
    
    # how I would actually do it
    lead.append(soup.find_all('div',{'class':'lead'})[0].text.strip())
    
    # sometimes you make wierd things - that works
    temp_3 = soup.find_all('time')
    temp_3 = temp_3[0]
    temp_3 = str(temp_3)[16:32]
    time_posted.append(temp_3)

In [ ]:
# h1 
soup.find_all('h1')

In [ ]:
soup.find_all('h1')[1]

In [ ]:
soup.find_all('h1')[1].text.strip()

In [ ]:
# lead
soup.find_all('div',{'class':'lead'})[0].text.strip()

In [ ]:
# time_posted
soup.find_all('time')[0]

#### Lastly, we put our collected information into a dataframe

In [ ]:
import pandas as pd
df = pd.DataFrame({'title':h1_list, 'lead':lead, 'time':time_posted})
df

In [ ]:
# saving df
df.to_csv('df_bold.dk.csv')

# loading df
pd.read_csv('df_bold.dk.csv', index_col=0)

# Exercise Set 7: Web Scraping 2

In this Exercise Set we shall develop our webscraping skills even further by practicing **parsing** and navigating html trees using `BeautifoulSoup` and furthermore train extracting information from raw text with no html tags to help, using regular expressions. 

But just as importantly you will get a chance to think about **data quality issues** and how to ensure reliability when curating your own webdata. 

## Exercise Section 7.1: Logging and data quality

> **Ex. 7.1.1:** *`Why` is it important to log processes in your data collection?*



> **Ex. 7.1.2:**
*`How` does logging help with both ensuring and documenting the quality of your data?*


## Exercise Section 7.2: Parsing a Table from HTML using BeautifulSoup.

In module_6 I showed you a neat little prepackaged function in pandas that did all the work. However today we should learn the mechanics of it. *(It is not just for educational purposes, sometimes the package will not do exactly as you want.)*

We hit the Basketball stats page from yesterday again: https://www.basketball-reference.com/leagues/NBA_2018.html.


> **Ex. 7.2.1:** Here we practice simply locating the table node of interest using the `find` method build into BeautifoulSoup. But first we have to fetch the HTML using the `requests` module. Parse the tree using `BeautifulSoup`. And then use the **>Inspector<** tool (* right click on the table < press inspect element *) in your browser to see how to locate the Eastern Conference table node - i.e. the *tag* name of the node, and maybe some defining *attributes*.

In [ ]:
import requests
url = 'https://www.basketball-reference.com/leagues/NBA_2018.html' # link to the website
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table_E = soup.findAll('table', {'id' : 'confs_standings_E'})

In [ ]:
table_E[0].text

You have located the table should now build a function that starts at a "table node" and parses the information, and outputs a pandas DataFrame. 

Inspect the element either within the notebook or through the **>Inspector<** tool and start to see how a table is written in html. Which tag names can be used to locate rows? How will you iterate through columns. Were is the header located?

> **Ex. 7.2.2:** First you parse the header which can be found in the canonical tag name: thead. 
Next you use the `find_all` method to search for the tag, and iterate through each of the elements extracting the text, using the `.text` method builtin to the the node object. Store the header values in a list container. 

> **Ex. 7.2.3:** Next you locate the rows, using the canonical tag name: tbody. And from here you search for all rows tags. Fiugre out the tag name yourself, inspecting the tbody node in python or using the **Inspector**. 

> **Ex. 7.2.4:** Next run through all the rows and extract each value, similar to how you extracted the header. However here is a slight variation: Since each value node can have a different tag depending on whether it is a digit or a string, you should use the `.children` method instead of the `.find_all` - (or write compile a regex that matches both the td tag and the th tag.) 
>Once the value nodes of each row has been located using the `.children` method you should extract the value. Store the extracted rows as a list of lists: ```[[val1,val2,...valk],...]```

In [228]:
from collections import defaultdict

def bb_func(table_soup):
    # Define empty lists
    header_list = []
    rows_list = []
    columns = []
    dict_bb = defaultdict(list)

    # Add the header to the list
    header_list.append(table_soup.findAll('th')[0].text)

    # Find all the columns and add to the columns list
    col = table_soup.find('thead').findAll('th', {'scope': 'col'})
    for i in range(len(col)):
        columns.append(col[i].text)
    
    # Add the rows to the row list
    row = table_soup.find('tbody').findAll('a')
    for i in range(len(row)):
        rows_list.append(row[i].text)
    
    # Get the row values
    values = table_soup.find('tbody').findAll('td')
    for i in range(len(values)):
        key = table_soup.find('tbody').findAll('td')[i]['data-stat']
        value = table_soup.find('tbody').findAll('td')[i].text
        dict_bb[key].append(value)

    # Make the DataFrame to return
    #new_cols = list(set(columns).difference(header_list))
    new_cols = columns[1:]
    df_bb = pd.DataFrame( data = dict_bb.values()).transpose()
    df_bb.index = rows_list
    df_bb.columns = new_cols

    return df_bb


In [229]:
df = bb_func(soup)
df

,W,L,W/L%,GB,PS/G,PA/G,SRS
Toronto Raptors,59,23,.720,—,111.7,103.9,7.29
Boston Celtics,55,27,.671,4.0,104.0,100.4,3.23
Philadelphia 76ers,52,30,.634,7.0,109.8,105.3,4.30
Cleveland Cavaliers,50,32,.610,9.0,110.9,109.9,0.59
Indiana Pacers,48,34,.585,11.0,105.6,104.2,1.18
Miami Heat,44,38,.537,15.0,103.4,102.9,0.15
Milwaukee Bucks,44,38,.537,15.0,106.5,106.8,-0.45
Washington Wizards,43,39,.524,16.0,106.6,106.0,0.53
Detroit Pistons,39,43,.476,20.0,103.8,103.9,-0.26
Charlotte Hornets,36,46,.439,23.0,108.2,108.0,0.07


> **Ex. 7.2.5:** Now locate all tables from the page, using the `.find_all` method searching for the table tag name. Iterate through the table nodes and apply the function created for parsing html tables. Store each table in a dictionary using the table name as key. The name is found by accessing the id attribute of each table node, using dictionary-style syntax - i.e. `table_node['id']`.

In [230]:
tables = soup.findAll('table', {'class' : 'suppress_all sortable stats_table'})
table_dict = {}
for i in range(len(tables)):
    table_dict[f'table{i}'] = bb_func(tables[i])

In [232]:
table_dict['table0']

,W,L,W/L%,GB,PS/G,PA/G,SRS
Toronto Raptors,59,23,.720,—,111.7,103.9,7.29
Boston Celtics,55,27,.671,4.0,104.0,100.4,3.23
Philadelphia 76ers,52,30,.634,7.0,109.8,105.3,4.30
Cleveland Cavaliers,50,32,.610,9.0,110.9,109.9,0.59
Indiana Pacers,48,34,.585,11.0,105.6,104.2,1.18
Miami Heat,44,38,.537,15.0,103.4,102.9,0.15
Milwaukee Bucks,44,38,.537,15.0,106.5,106.8,-0.45
Washington Wizards,43,39,.524,16.0,106.6,106.0,0.53
Detroit Pistons,39,43,.476,20.0,103.8,103.9,-0.26
Charlotte Hornets,36,46,.439,23.0,108.2,108.0,0.07


> **Ex. 7.2.6. (extra) :** Compare your results to the pandas implementation [pd.read_html](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_html.html)

## Exercise Section 7.3: The European Research Counsel (optional)
**NOTE** Exercise 7.3 is difficult and therefore also optional. I expect less than 10% of you being able to solve this one.

Imagine we wanted to analyze whether the European funding behaviour was biased towards certain countries and gender. We might decide to scrape who has received funding from the ERC.
https://erc.europa.eu/

* First we figure find navigate the grant listings.
* Next we figure out how to page these results. 
* And finally we want to grab the information.

### Data Storage and operating system interactions

> **Ex. 7.3.1:** *Import the python library `os`. Write pyhon code in this jupyter notebook creating a new folder in your directory called "erc_funding". Inside your new folder create 3 subfolders called 'mapping', 'raw_data' and 'parsed_data'.*



### Mapping

> **Ex. 7.3.2:** *Investigate [https://erc.europa.eu/projects-figures/erc-funded-projects/results?items_per_page=100&search_api_views_fulltext=&](https://erc.europa.eu/projects-figures/erc-funded-projects/results?items_per_page=100&search_api_views_fulltext=&). Figure out how many sites you need to loop thorugh. Save the response for each site using in `condecs` in your "mapping" subfolder. Use the `tqdm` to track your loop.
Use the Snorre Ralund Connector class to log your activity.*



### Parsing

> **Ex. 7.3.3:** *Write a function that takes a filename (from our mapping subfolder) as and input and returns (in our parsed_data subfolder) a `pandas`dataframe of parsed information. Use `os` library to navigate your operating system (paths) and `condecs` library to read files inside your function. Last, concatenate all your dataframes into one dataframe you call "df" consisting of all parsed data.*

### Reliability and Data Quality

##### Inspect the data
> **Ex. 7.3.4:** *Investigate your dataframe "df". Check for dublicates. Count NaN values. Create a `matplotlib` histrogram plot for every column of "df" illustrating the lenght of the string (x-axis) and row count( y-axis).*

##### Do simple descriptives
> **Ex. 7.3.5:** *Create a value_counts() for each of the three columns (Host Institution (HI), Researcher (PI) and Project acronym) in your "df". What can counting do for us in this exercise in terms of Reliability and Data Quality?*

##### Visualize the Log
> **Ex. 7.3.6:** *Load your "erc_log.csv". Convert the time column 't' to datetime. Use `matplotlib` to create three plots: (1) time it took to make the call, (2) the response size over time, and (3) the delta_t against the response_size .)*